# 目标检测

参考: [intermediate/detection](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html?highlight=detection)

在本教程中，将在 [Penn-Fudan 数据库](https://www.cis.upenn.edu/~jshi/ped_html/) 中对预训练的 [Mask R-CNN 模型](https://arxiv.org/abs/1703.06870) 进行微调，用于行人检测和分割。它包含 170 张图像和 345 个行人实例，将用它来说明如何使用torchvision中的新功能，以便在自定义数据集上训练实例分割模型。

## 定义数据集

训练目标检测、实例分割和行人关键点检测的参考脚本允许轻松支持添加新的自定义数据集。数据集应该继承标准 {class}`~torch.utils.data.Dataset` 类，并实现 `__len__` 和 `__getitem__`。

唯一需要的指定是数据集 `__getitem__` 应该返回:

- `image`: `(H, W)` 的 {class}`PIL.Image` 对象
- `target`: 包含下列字段的字典
    - `boxes` (`FloatTensor[N, 4]`): `N` 个边界框的 `[x0, y0, x1, y1]` 格式的坐标，取值范围为 `0` ~ `W` 和 `0` ~ `H`
    - `labels` (`Int64Tensor[N]`): 每个边界框的标签。`0` 总是表示背景类
    - `image_id` (`Int64Tensor[1]`): 图像标识符。它在数据集中的所有图像之间应该是唯一的，并在评估期间使用
    - `area` (`Tensor[N]`): 边界框的面积。这是在评估 COCO 指标时使用的，用于将度量分数区分为小、中、大边界框。
    - `iscrowd` (`UInt8Tensor[N]`): 带有 `iscrowd=True` 的实例将在评估时被忽略。
    - (可选) `masks` (`UInt8Tensor[N, H, W]`): 每个对象的分割掩码
    - (可选) `keypoints` (`FloatTensor[N, K, 3]`): 对于 `N` 个对象中的每一个，它包含了 `[x, y, visibility]` 格式的 `K` 个关键点，定义了对象。`visibility = 0` 表示关键点不可见。注意，对于数据增强，flipping 关键点的概念依赖于数据表示，您可能应该为新的关键点表示调整 `references/detection/transforms.py`

如果您的模型返回上述方法，它们将使其既能用于训练又能用于评估，并将使用来自 `pycocotools` 的评估脚本，这些脚本可以通过 `pip install pycocotools` 安装。

````{note}
对于 Windows，请使用命令从 [gautamchitnis](https://github.com/gautamchitnis/cocoapi) 安装 `pycocotools`

```bash
pip install git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI
```
````

```{tip}
`labels` 上要注意一点: 模型将类 `0` 作为背景。如果数据集不包含背景类，则标签中不应该有 `0`。例如，假设您只有两个类，猫和狗，您可以定义`1` (不是 `0`)来表示猫，定义 `2` 来表示狗。因此，例如，如果图像具有两个类，那么你的标签张量应该是 `[1, 2]`。
```

此外，如果您想在训练期间使用纵横比(aspect ratios)分组(以便每批只包含具有相似纵横比的图像)，那么建议还实现 `get_highight_and_width` 方法，该方法返回图像的高度和宽度。

## 为 PennFudan 编写自定义数据集

[下载并解压压缩文件](https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip) 后，有如下的文件夹结构:
```
PennFudanPed/
  PedMasks/
    FudanPed00001_mask.png
    FudanPed00002_mask.png
    FudanPed00003_mask.png
    FudanPed00004_mask.png
    ...
  PNGImages/
    FudanPed00001.png
    FudanPed00002.png
    FudanPed00003.png
    FudanPed00004.png
```

接口实现:

In [1]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

下面定义可以在这个数据集上执行预测的模型。

## 定义模型

在本教程中，将使用 [Mask R-CNN](https://arxiv.org/abs/1703.06870)，它是基于 [Faster R-CNN](https://arxiv.org/abs/1506.01497) 的 top。Faster R-CNN 可以预测图像中潜在对象的边界框和类别分数。
![](images/tv_image03.png)
Mask R-CNN 在Faster R-CNN 中添加了额外的分支，它还预测每个实例的分割掩码。
![](images/tv_image04.png)

有两种常见的情况，您可能想要修改 torchvision modelzoo 中的可用模型。第一个是当想要从预先训练好的模型开始，只是对最后一层进行微调。另一种情况是当想要用不同的模型来替代模型的主干时(例如，为了更快的预测)。

### 1 对预训练模型进行微调

假设希望从预训练过 COCO 的模型开始，并希望针对特定的类对其进行微调。可行的方法:

In [2]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/ai/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

### 2 修改模型以添加不同的主干

In [3]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /home/ai/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

### 基于 PennFudan 数据集的实例分割模型

在我们的例子中，希望对预训练的模型进行微调，因为数据集非常小，所以将遵循方法 1。

这里还想计算实例分割掩码，所以将使用 Mask R-CNN:

In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

就是这样，这将使模型准备好在您的自定义数据集上进行训练和评估。

## 把每个组件组合在一起

在 [references/detection](https://github.com/pytorch/vision/blob/main/references/detection/) 中，有许多辅助函数来简化训练和评估检测模型。在这里，将使用 `references/detection/engine.py`、`references/detection/utils.py` 和 `references/detection/transforms.py`。只需复制 `references/detection` 下的所有内容到您的文件夹中并在这里使用。

让我们写一些数据增强/变换的辅助函数:

In [9]:
import references.detection.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

## 测试 {meth}`forward` 方法(可选)

在对数据集进行迭代之前，最好看看模型在样本数据的训练和推断期间期望得到什么。

In [11]:
from references.detection.utils import collate_fn
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
dataset = PennFudanDataset('/media/pc/data/lxw/data/PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=collate_fn)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

编写执行训练和验证的主函数:

In [18]:
from references.detection.engine import train_one_epoch, evaluate
from references.detection.utils import collate_fn


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PennFudanDataset('/media/pc/data/lxw/data/PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('/media/pc/data/lxw/data/PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

运行:

In [19]:
main()

Epoch: [0]  [ 0/60]  eta: 0:03:02  lr: 0.000090  loss: 3.2570 (3.2570)  loss_classifier: 0.5470 (0.5470)  loss_box_reg: 0.2642 (0.2642)  loss_mask: 2.4101 (2.4101)  loss_objectness: 0.0332 (0.0332)  loss_rpn_box_reg: 0.0024 (0.0024)  time: 3.0390  data: 0.5157  max mem: 1857
Epoch: [0]  [10/60]  eta: 0:00:21  lr: 0.000936  loss: 1.7240 (2.0881)  loss_classifier: 0.4040 (0.3690)  loss_box_reg: 0.2647 (0.2820)  loss_mask: 1.0286 (1.4086)  loss_objectness: 0.0265 (0.0241)  loss_rpn_box_reg: 0.0031 (0.0044)  time: 0.4307  data: 0.0531  max mem: 2856
Epoch: [0]  [20/60]  eta: 0:00:12  lr: 0.001783  loss: 0.8028 (1.3965)  loss_classifier: 0.1938 (0.2584)  loss_box_reg: 0.2647 (0.2628)  loss_mask: 0.3001 (0.8529)  loss_objectness: 0.0118 (0.0181)  loss_rpn_box_reg: 0.0031 (0.0044)  time: 0.1700  data: 0.0070  max mem: 3153
Epoch: [0]  [30/60]  eta: 0:00:07  lr: 0.002629  loss: 0.5896 (1.1199)  loss_classifier: 0.0944 (0.2006)  loss_box_reg: 0.2132 (0.2531)  loss_mask: 0.2183 (0.6455)  loss_ob